In [1]:
import pandas as pd
import numpy as np

In [2]:
# reading csv's
school_data = pd.read_csv('Resources/schools_complete.csv')
student_data = pd.read_csv('Resources/students_complete.csv')
school_df = pd.merge(student_data,school_data,how ='left',on='school_name')
#total no of schools
school_total = school_df.school_name.value_counts().count()
#total no of students
student_total = school_df["Student ID"].count()
# total budget
#-changed it at the end instead---total_budget= school_data.budget.sum().astype(str)
total_budget= school_data.budget.sum()
#Average math score
avg_math = school_df.math_score.mean()
#Average reading score
avg_read= school_df.reading_score.mean()
#overall passing rate
overall_pass = (school_df.math_score.mean() + school_df.reading_score.mean())/2
# percentage of student with passing math score 70 or greater
new_df = (school_df.loc[school_df.math_score >=70])
per_math=new_df.student_name.count()/school_df.student_name.count()*100
 # percentage of student with passing reading score 70 or greater
new_df = (school_df.loc[school_df.reading_score >=70])
per_reading=new_df.student_name.count()/school_df.student_name.count()*100 
#District DataFrame
district_df =pd.DataFrame(columns =['Total\n Schools','Total\nStudents','Total Budget','Average\nMath\nScore','Average\nReading\nScore','%\nPassing\nMath','%\nPassing\nReading','%\nOverall\Passing'],data=[[school_total,student_total,total_budget,avg_math,avg_read,per_math,per_reading,overall_pass]])
district_df['Total Budget']='$' + district_df['Total Budget'].map('{:,}'.format)
district_df

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall\Passing
0,15,39170,"$24,649,428",78.985371,81.87784,74.980853,85.805463,80.431606


In [3]:
# grouped by school name  
new_df = school_df.groupby('school_name')
# create a dataframe and count no of students
school_summary = pd.DataFrame(new_df['student_name'].count())
# creating dateframe for school_type
school_type =pd.DataFrame(new_df['type'].unique())
#student count in a variable for computation
student_count=new_df.student_name.count()
#merging to create dataframe with type
school_summary1 = pd.merge(school_summary,school_type,how='left', on='school_name')
# total school budget
budget_df = pd.DataFrame(new_df['budget'].unique())
school_summary = pd.merge(school_summary1,budget_df,how='left',on = 'school_name')
#budget per student
total_sc_budget_per = new_df['budget'].unique().values/student_count
#converting series to dataframe
total_sc_df = total_sc_budget_per.to_frame()
school_summary2 = pd.merge(school_summary,total_sc_df,how ='left',on='school_name')
school_summary2.rename(columns={'student_name_x':'Total\nStudents','student_name_y':'Per\nStudent\nBudget'},inplace = True)
#calculate average math score
avg_math=school_df.groupby('school_name')['math_score'].mean()
#converting to dateframe
new_avg_math=avg_math.to_frame()
new_avg_math.rename(columns={'math_score':'Average\nMath\nScore'},inplace=True)
school_summary3 = pd.merge(school_summary2,new_avg_math,how ='left',on='school_name')
#calculate average reading score
avg_reading=school_df.groupby('school_name')['reading_score'].mean()
#converting to dateframe
new_avg_reading=avg_reading.to_frame()
new_avg_reading.rename(columns={'reading_score':'Average\nReading\nScore'},inplace = True)
school_summary4 = pd.merge(school_summary3,new_avg_reading,how ='left',on='school_name')
#computing pass % math
math_stu_pass= school_df[school_df['math_score']>= 70].groupby('school_name')['student_name'].count()
total_stu_school=school_df.groupby('school_name')['student_name'].count()
math_per_pass = math_stu_pass/total_stu_school*100
math_per_pass_df = math_per_pass.to_frame()
#change column name before merging
math_per_pass_df.rename(columns={'student_name':'%Passing\nMath'},inplace =True)
school_summary5=pd.merge(school_summary4,math_per_pass_df,how='left',on='school_name')
#computing pass % reading
reading_stu_pass= school_df[school_df['reading_score']>= 70].groupby('school_name')['student_name'].count()
reading_per_pass = reading_stu_pass/total_stu_school*100
reading_per_pass_df = reading_per_pass.to_frame()
# rename the column before merging
reading_per_pass_df.rename(columns={'student_name':'%Passing\nReading'},inplace =True)
school_summary6=pd.merge(school_summary5,reading_per_pass_df,how='left',on='school_name')
#overall passing rate
over_pass_rate =(math_per_pass+reading_per_pass)/2
#convert series to dataframe
overall_pass_rate=over_pass_rate.to_frame()
#change column name
overall_pass_rate.rename(columns={'student_name':'Overall Pass\n Rate'},inplace = True)
school_summary7=pd.merge(school_summary6,overall_pass_rate,how='left',on ='school_name')
#cleaning the data(removing brackets)
school_summary7['type']=school_summary7['type'].str.get(0)
school_summary7['budget']=school_summary7['budget'].str.get(0)
#school_summary7['budget'] = '$'+ school_summary7['budget'].map('{:,}'.Format)
school_summary7.rename(columns={'type':'School\nType','budget':'Total\nSchool\nBudget'},inplace=True)
school_summary7['Per\nStudent\nBudget']=school_summary7['Per\nStudent\nBudget'].str.get(0)
school_summary7=school_summary7.sort_values(by=['Overall Pass\n Rate'],ascending = False)

#top performing
school_summary7.head(5)

,Total Students,School Type,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,%Passing Math,%Passing Reading,Overall Pass Rate
school_name,,,,,,,,,
Cabrera High School,1858,Charter,1081356,582.0,83.061895,83.975780,94.133477,97.039828,95.586652
Thomas High School,1635,Charter,1043130,638.0,83.418349,83.848930,93.272171,97.308869,95.290520
Pena High School,962,Charter,585858,609.0,83.839917,84.044699,94.594595,95.945946,95.270270
Griffin High School,1468,Charter,917500,625.0,83.351499,83.816757,93.392371,97.138965,95.265668
Wilson High School,2283,Charter,1319574,578.0,83.274201,83.989488,93.867718,96.539641,95.203679


In [4]:
#bottom performing
school_summary7.tail(5)

,Total Students,School Type,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,%Passing Math,%Passing Reading,Overall Pass Rate
school_name,,,,,,,,,
Ford High School,2739,District,1763916,644.0,77.102592,80.746258,68.309602,79.299014,73.804308
Johnson High School,4761,District,3094650,650.0,77.072464,80.966394,66.057551,81.222432,73.639992
Huang High School,2917,District,1910635,655.0,76.629414,81.182722,65.683922,81.316421,73.500171
Figueroa High School,2949,District,1884411,639.0,76.711767,81.158020,65.988471,80.739234,73.363852
Rodriguez High School,3999,District,2547363,637.0,76.842711,80.744686,66.366592,80.220055,73.293323


In [5]:
#dropping columns not needed for next section
m_data = student_data.drop(['reading_score','gender','Student ID'],axis = 1)
#creating series to dataframe (math)for 9th - 12th grade and changing the column names
math_9th = m_data[m_data['grade']=='9th'].groupby(['school_name'])['math_score'].mean()
math_9df= math_9th.to_frame()
math_9df.rename(columns={'math_score':'9th'},inplace = True)
math_10th = m_data[m_data['grade']=='10th'].groupby(['school_name'])['math_score'].mean()
math_10df= math_10th.to_frame()
math_10df.rename(columns={'math_score':'10th'},inplace = True)
math_11th = m_data[m_data['grade']=='11th'].groupby(['school_name'])['math_score'].mean()
math_11df= math_11th.to_frame()
math_11df.rename(columns={'math_score':'11th'},inplace = True)
math_12th = m_data[m_data['grade']=='12th'].groupby(['school_name'])['math_score'].mean()
math_12df= math_12th.to_frame()
math_12df.rename(columns={'math_score':'12th'},inplace = True)
new_math_group = m_data.groupby(m_data['school_name'])
new_math_df=pd.DataFrame(new_math_group['student_name'].count())
new_math_df1 = pd.merge(new_math_df,math_9df,how='left',on='school_name')
new_math_df2 = pd.merge(new_math_df1,math_10df,how='left',on='school_name')
new_math_df3 = pd.merge(new_math_df2,math_11df,how='left',on='school_name')
new_math_df3 = pd.merge(new_math_df3,math_12df,how='left',on='school_name')
math_summary= new_math_df3.drop(['student_name'],axis=1)
#math summary by grade
math_summary

,9th,10th,11th,12th
school_name,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248


In [6]:
#creating series for Reading to dataframe for 9th - 12th grade and changing the column names

read_9th = student_data[student_data['grade']=='9th'].groupby(['school_name'])['reading_score'].mean()
read_9df= read_9th.to_frame()
read_9df.rename(columns={'reading_score':'9th'},inplace = True)
read_10th = student_data[student_data['grade']=='10th'].groupby(['school_name'])['reading_score'].mean()
read_10df= read_10th.to_frame()
read_10df.rename(columns={'reading_score':'10th'},inplace = True)
read_11th = student_data[student_data['grade']=='11th'].groupby(['school_name'])['reading_score'].mean()
read_11df= read_11th.to_frame()
read_11df.rename(columns={'reading_score':'11th'},inplace = True)
read_12th = student_data[student_data['grade']=='12th'].groupby(['school_name'])['reading_score'].mean()
read_12df= read_12th.to_frame()
read_12df.rename(columns={'reading_score':'12th'},inplace = True)
new_read_group = student_data.groupby(student_data['school_name'])
new_read_df=pd.DataFrame(new_read_group['student_name'].count())
new_read_df1 = pd.merge(new_read_df,read_9df,how='left',on='school_name')
new_read_df2 = pd.merge(new_read_df1,read_10df,how='left',on='school_name')
new_read_df3 = pd.merge(new_read_df2,read_11df,how='left',on='school_name')
new_read_df3 = pd.merge(new_read_df3,read_12df,how='left',on='school_name')
read_summary= new_read_df3.drop(['student_name'],axis=1)
# reading Summary by grades
read_summary


,9th,10th,11th,12th
school_name,,,,
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564


In [7]:
#binning by scores
new_bin=school_summary7.reset_index()
score_bin=new_bin.drop(['school_name','Total\nStudents','School\nType','Total\nSchool\nBudget'],axis= 1)
#creating bins
bins =[0,580,600,620,640,660]
#creating names for the bin
groups =["<$580","$580-600","$600-620","$620-640","$640-660"]
score_bin["Spending Ranges\n(Per Student)"] =pd.cut(score_bin['Per\nStudent\nBudget'],bins,labels=groups)
score_bin = score_bin.groupby(score_bin["Spending Ranges\n(Per Student)"])

score_bin.max()


,Per Student Budget,Average Math Score,Average Reading Score,%Passing Math,%Passing Reading,Overall Pass Rate
Spending Ranges (Per Student),,,,,,
<$580,578.0,83.274201,83.989488,93.867718,96.539641,95.203679
$580-600,600.0,83.803279,83.975780,94.133477,97.039828,95.586652
$600-620,609.0,83.839917,84.044699,94.594595,95.945946,95.270270
$620-640,639.0,83.418349,83.848930,93.392371,97.308869,95.290520
$640-660,655.0,77.289752,81.182722,68.309602,81.316421,73.807983


In [8]:
#Scores by school size
school_size =pd.DataFrame(new_df['size'].unique())
school_size_df=pd.merge(school_summary7,school_size,how='left',on='school_name')
size_bin=school_size_df.reset_index()
newsize_bin=size_bin.drop(['school_name','Total\nStudents','School\nType','Total\nSchool\nBudget','Per\nStudent\nBudget'],axis= 1)
#creating bins
bins=[0,1000,2000,3000,4000,5000]
groups=["<1000","1000-2000","2000-3000","3000-4000","4000-5000"]
newsize_bin["Scores by\nSchool Size"]=pd.cut(newsize_bin['size'],bins,labels=groups)
newsize_bin=newsize_bin.groupby(newsize_bin["Scores by\nSchool Size"])
newsize_bin.max()

,Average Math Score,Average Reading Score,%Passing Math,%Passing Reading,Overall Pass Rate,size
Scores by School Size,,,,,,
<1000,83.839917,84.044699,94.594595,96.252927,95.270270,962
1000-2000,83.682222,83.975780,94.133477,97.308869,95.586652,1858
2000-3000,83.274201,83.989488,93.867718,96.539641,95.203679,2949
3000-4000,76.842711,80.744686,66.366592,80.220055,73.293323,3999
4000-5000,77.289752,81.033963,66.752967,81.933280,74.306672,4976


In [9]:
#Score by school Type
type_bin=new_bin.drop(['school_name','Total\nStudents','Total\nSchool\nBudget','Per\nStudent\nBudget'],axis= 1)

In [10]:
new_type_df=type_bin.groupby(['School\nType'])['Average\nMath\nScore','Average\nReading\nScore','%Passing\nMath','%Passing\nReading','Overall Pass\n Rate'].mean()
#school type
new_type_df

,Average Math Score,Average Reading Score,%Passing Math,%Passing Reading,Overall Pass Rate
School Type,,,,,
Charter,83.473852,83.896421,93.620830,96.586489,95.103660
District,76.956733,80.966636,66.548453,80.799062,73.673757
